# Objects and object-oriented programming

A fundamental concept of object (-oriented) programming (OOP) is the **object**---a coherent and complete combination of *data and operations* on these data.
Objects usually (not always) hide data and externally provide only a limited set of operations, i.e. *methods*.
Typically the person who accesses an object is not interested in the implementation of those methods
(the object can silently delegate their execution to other objects), or the way the objects's data are arranged.

Object-oriented programming also includes several other concepts. Programming languages adopt and interpret them in different ways. Let us show which concepts are present in Python and how to use them.

<!-- TEASER_END -->

## Objects

In Python, *everything is an object* (unlike C++ or Java). This holds for all built-in types (numbers, strings, ...), all containers,
as well as functions, modules, and object types. Absolutely everything provides some methods.

### Object's type

Each object has a type; types can be divided (although this division does not have many practical implications)
into *built-in types* (list, tuple, int, ...) and *classes* (types defined using the `class` keyword). 
The type determines what methods an object offers, it is a sort of a template (general characteristics), from which the individual object differs by its internal state (specific properties). 
We say that *object is an instance of that type (class)*. 
To determine the type of an object, Python has a built-in function `type`.

In [ ]:
print(type("Babička"))
print(type(46878678676848648486))              # int
print(type(list()))                            # an instance of the list type
print(type(list))                              # the list type itself

`isinstance` checks the object type:

In [ ]:
print(isinstance([1, 2], list))

### Instantiation

An instance of a given type is created similarly to calling a function. If we have a data type (class),
we create an instance, just as if we wanted to call it,
i.e. using paretheses. After all, we have already done so with built-in types like `tuple`, `dict` or `list`. Effectively, the instantiation process involves
calling the class constructor (see below).

In [ ]:
object1 = list()        # Creates a new instace of the list type
object2 = list          # This does not create a new instance! It just gives a new name to the list type

# Let's see what we've got
print("object1 =", object1)
print("object2 =", object2)

In [ ]:
# Now we can create a list using object2
object2()

*Note: This is roughly similar to using C++'s or Java's `new` operator*.

### Using methods

Method is a function that is tied to some object (and is basically meaningless without the object) and operates with its data. It can also change the internal state of the object, i.e. the attribute values.

In Python, methods are called using dot notation, **`object.method(arguments)`**

In [ ]:
numbers = [45, 46, 47, 48]     # numbers is a list instance
numbers.append(49)             # we call its append method

numbers

The `append` method has no meaning in itself, only in conjunction with a specific list; it adds a new element to the list.

## Classes

Historically, there was a difference between the basic built-in *types* and user *classes*. Nowadays, the two terms have practically
merged (and they are interchangeable) though when creating new types, we are more likely to refer to them as *classes* and we use the `class` keyword to start
a new type definition. Like built-in types, the user-define ones have methods and data (attributes), which we can arbitrarily define.

The simplest definition of an empty class (`pass` is used for empty classes and methods to play well with the indentation rules):

In [ ]:
class MyClass:    # create a new class called MyClass
    pass          # the class is empty (but implicitly inherits everything from type "object")

### Method definition

Methods are defined within the `class` block. (*Actually, methods can be added to the class later, but it is not the preferred method.*)

Conventional methods (instance methods) are called on a particular object. Besides, there are also so-called class methods and static methods, which we are not going to discuss.

Quite unusual (unlike C++, Java and other languages) is that the first argument of the method is the object on which the method is called.
Without that, the method does not know with which object it is working! By convention (which is perhaps never violated),
this first argument is called **`self`**. When the method is called, Python fills this argument automatically.


In [ ]:
class Car:
    def roll(self, distance):     # Don't forget *self*
        print(f"Rolling {distance} kilometers.")
        
car = Car()                        
car.roll(100)                     # self is omitted

Passing the first argument explicitly is an error! Notice the number of arguments that Python complains about.

In [ ]:
car.roll(car, 100)

**Exercise:** Add a `honk` method to the `Car` class that accepts an optional argument `count`
(with a default value 1 to represent a short honk) and prints the "honk" string `count` times.

In [ ]:
class Car:
    def roll(self, distance):     # Don't forget *self*
        print(f"Rolling {distance} kilometers.")

    # ...
        
car = Car()                        
car.honk(5) 

### Constructor

Contructor is the method that initializes the object. It's called when we create a new instance.
We can (and in most cases we do) define it but we do not have to, in which case the default constructor is used that simply does nothing (special).
The constructor in Python is always named **`__init__`** (two underscores before and after).

In [ ]:
class MyClass2:
    def __init__(self):
        print("We are in the constructor")

print("Before instantiating MyClass2")
# The constructor will be called now
obj = MyClass2()
print("After instantiating MyClass2")

### Attributes

Python does not distinguish between methods and data (such as in general the variables - everything is an object). Everything is an **attribute** of the object. 
Values are assigned similarly to variables but we have to add the object and the dot. 
Attributes may not even exist yet when they are assigned (they do not have to be declared).
The **convention** you should obey is to create all attributes inside the `__init__` method.

(NB. Internally attributes are stored in dictionaries and access to them is through the dictionary of the object itself, its class, its parent class, ...). 

In [ ]:
class Car:
    def __init__(self, consumption):    # constructor with an argument
        self.consumption = consumption  # simply store as an attribute (of self)
    
    def roll(self, distance):
        # the consumption attribute is used
        gas = distance / 100 * self.consumption
        # gas is local, not an attribute
        print(f"Rolling {distance} kilometrs, using {gas} liters of gas.")
        
car = Car(15)
print(f"My car has a consumption of {car.consumption} l/100 km.")  
car.roll(150)

The list of all attributes is returned by `dir`. 
Note that we list only the public attributes, see "The underscore convention" below for an explanation.

In [ ]:
# attributes starting with underscore(s) are special, we'll filter them out
", ".join(item for item in dir(car) if not item.startswith("_"))  

### Properties

Properties are "smarter" data. They allow you to step into the process of reading or setting attributes.
It is useful, for example, if an object has several interdependent parameters, and we do not want to store them independently;
or if we want to check what value is stored; or if we want to do anything interesting with the values.
Properties are also handy for lazy evaluation - i.e. for postponing some operation (calculation, resource access etc.)
to the point when it is actually needed.

From the syntactic point of view, we must first define the method that bears the name of the property and that "reads" the property (returns its value).
The line above must include a **`property`** *decorator* 
(for details see e.g. [Primer on Python Decorators](https://realpython.com/primer-on-python-decorators/)). 
If we want, we can then create methods for writing and deleting.

Once we have created the following properties, we approach them as common data attributes - call them without brackets and assign to them using the sign "equals".

Properties work like properties in C# or Java JavaBeans. However, notice that for accessing properties exactly the same notation as for accessing data
attributes is used. Hence if someone wants to change the behavior of a data attribute and make it a property,
clients of the class will not recognize it and will not have to make any changes in the code.
It is therefore not necessary and even advisalble to pro-actively create trivial properties that encapsulate only access to attributes
(like we would certainly do in Java).

We will show how properties work on a simple example of a Circle class, which can set both the radius and the area consistently.

In [ ]:
import math
import numbers


class Circle:
    def __init__(self, r):
        self.radius = r
        
    @property                          # this will be our"reader"
    def area(self):                    # this looks like any other method
        return math.pi * self.radius ** 2
    
    @area.setter                       # area "setter"
    def area(self, s):                 
        print(f"Changing the area to {s}")
        if not isinstance(s, numbers.Number):   # is s a number?
            raise TypeError("Area must be a number")
        # the radius must be set consistently
        self.radius = math.sqrt(s / math.pi)
        
    @area.deleter
    def area(self):
        raise AttributeError("Deleting circle's area does not make any sense")
    
# create a circle with unity radius
circle = Circle(1)
print(f"r = {circle.radius}")    # usual attribute
print(f"S = {circle.area}")      # a property

circle.area = 5                  # Changing radius using the area setter
print(f"r = {circle.radius}")    # We've changed the radius accordingly
print(f"S = {circle.area}")      # a property

In [ ]:
# Let's see if the check in the "setter" works
circle.area = "Just like the biggest Czech pond, which is called Rožmberk."

In [ ]:
# Another meaningless operation 
del circle.area

**Exercise:** Take the `Car` class definition above and add a property `miles_per_gallon`
(assuming 1 mile = 1.609 km, 1 gallon = 3.785 l) to make the car consumption understandable for U.S. users.

**Bonus task:** Make this property writable, correctly updating the underlying "consumption" attribute

### Encapsulation 

Python does not adhere to this (fundamental) OOP concept very strongly. The principles of OOP claim that the data should not be accessible from outside. Other languages usually offer a way of hiding some methods (such as the keywords private or protected in C++, Java). Python does not try to resolve this issue and, by default, everything is accessible. 

Instead, there exist the following conventions:

* Object's data are not modified from outside (unless the class is really primitive or is explicitly designed for this).
* Methods whose names start with an underscore are not called from outside (they are not part of the "public" interface).
* To protect data, we can make them properties.
* Any differences in general and the way in which the methods and data are handled should be included in the class documentation.
* There are ways you can enforce encapsulation (redefining the access to attributes, ...) but those are rarely used (and rarely really useful).

In return, Python offers a very high level of **introspection**, or the ability to learn information about objects (their type, attributes, etc.) at runtime.


#### The underscore convention

In Python conventions are generally very strongly entrenched. It is perhaps the most visible in the context of objects.

1. "Private" attributes (attributes in Python often means both data and methods - everything is an object) are named with an underscore at the beginning, e.g \_private_method.
2. Two underscores at the beginning of the name of an attribute renames it so it's really hard to reference the attribute outside the context of the class.
Generally, double leading underscores should be used only to avoid name conflicts with attributes in classes designed to be subclassed.
3. Attributes with two undescores at the beginning and at the end have a special meaning (see [documentation](https://docs.python.org/3/reference/datamodel.html#special-method-names)). 
We have already seen __init__  and will look at several others.
    * `__repr__` and `__str__` convert the object to a string.
    * `__getattr__` and `__setattr__` are used for reading and storing not found attributes.
    * `__call__` will be called when we use the object as a function.
    * `__doc__` contains documentation (docstring).
    * `__dict__` contains the dictionary with the namespace of the object.
    * ... Furthermore, there are special features for logical operators, to emulate the functionality of containers (iteration, items, cuts), for arithmetic operations, etc.


In [ ]:
# what an instance of the object type contains?
dir(object())

In [ ]:
# and a simple function?
def foo(x):
    """This is function foo"""
    return x

dir(foo)

## Inheritance 

A class can inherit (derive) its behavior (and data) from another class (or multiple classes), 
thus saving a lot of work in the repetition of common features. 
In this case, we say that our new class (child or subclass) inherits from the original (parent) class(es).

* In a subclass, we can change the definition of some of the methods of the superclass.
* Constructors are inherited by default (unlike C++ or Java, in Python we have to explicitly call the superclass constructor only if we define a new constructor).
* Subclasses can be used wherever the parent class(es) can be used. *This applies even more generally in Python as we usually do not check the specific type. Instead, we look for particular attributes / methods. This is easily possible because Python is dynamically typed.*

**Syntax:** The name of the parent class is given in parentheses after the name (By default, all classes inherit from `object` which you may but need not include).

In [ ]:
class Human(object):
    def __init__(self, name):          # The constructor sets the name
        self.name = name
    
    def say(self, what):               # The default say method
        print(type(self).__name__ + ": " + what)
    
    def introduce(self):             
        self.say(f"My name is {self.name}.")
        
    def greet(self):                 
        self.say("Hello!")
        
    def goodbye(self):
        self.say("Good bye!")        
    
    
class Serviceman(Human):
    def repair_tv(self):         # A new method
        self.say("Give me 5 minutes.")
        print("---The serviceman is working.---")
        self.say("Done.")
        
    def introduce(self):            # introduce differently; self.name is used here
        self.say(f"I'm {self.name}.")

                
class Patient(Human):
    def say(self, what):            # redefined method
        """Say something with a running nose."""
        trantab = "".maketrans("nmNM", "dbDB")
        Human.say(self, what.translate(trantab))   # call parent class' method
        self.sneeze()
        
    def sneeze(self):                 # A new method - other humans do not sneeze
        print("---Achoo---")
    

joe = Serviceman("Joe Smith")
bill = Patient("Bill Jones")

# A daily conversation
joe.greet()
bill.greet()
joe.introduce()
bill.introduce()
bill.say("Can you fix my TV, please?")
joe.repair_tv()
bill.say("Thank you very much.")
joe.goodbye()
bill.goodbye()

In [ ]:
bill.repair_tv()           # Patients do not repair TV's

A sick electrician could be created using multiple inheritance, in which case we would have to consider if parent methods are called properly. Even better, we could use so-called mix-ins and inject properties into objects dynamically, but this is really an advanced topic that we will not cover.


**Exercise:** Make a `Trabant` (or `Wartburg` or `Skoda105` or `Maluch`...) class
that inherits from `Car` but has two additional specifics:

- The engine cannot `roll` more than 50 kilometers. If you try to call the methods with a larger distance,
it prints "The car broke down."

- The honker is broken: Instead of "honk", it prints "khkhrkhrxueeeeee".

### Inheriting from built-in types 
Classes can also inherit from built-in types. 
This is often useful (unlike our very unuseful example below).

In [ ]:
# A list that does not return its item unless pleaded
class PeevishList(list):
    def __getitem__(self, index):                     # redefining the method that handles getting items by [...]
        if isinstance(index, tuple) and index[1].lower()[:6] in ("please"):
            return list.__getitem__(self, index[0])   # the parent's method
        else:
            print("What about pleading?")
            return None
        
s = PeevishList((1, 2, 3, 4))
print(s[1])


In [ ]:
print(s[1, "please"])

Some types should not be directly subclassed though. `list` is one of those, because it is easy to create
an incosistent behaviour: the `pop` method works without pleading, which is likely not expected.

In [ ]:
s.pop()

## Advanced topics 

The following advanced topics are very interesting and terribly useful 
but we do not have enough time to explain them here. 
However, we recommend reading about them, if you have a little time.

* Multiple inheritance
* Class methods
* Static methods
* Abstract classes
* Polymorphism
* Metatclasses
* Design Patterns


Some resources on these topics:
* https://realpython.com/python3-object-oriented-programming/
* https://python-textbok.readthedocs.io/en/1.0/Object_Oriented_Programming.html
* https://www.digitalocean.com/community/conceptual_articles/s-o-l-i-d-the-first-five-principles-of-object-oriented-design
* https://python-patterns.guide/
* https://github.com/faif/python-patterns
* https://refactoring.guru/design-patterns